# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'07-10-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'07-10-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-07-11 04:21:43,33.93911,67.709953,131586,5561,80226.0,45799.0,Afghanistan,338.021083,4.226133
1,NaN,NaN,NaN,Albania,2021-07-11 04:21:43,41.15330,20.168300,132587,2456,130047.0,84.0,Albania,4607.234693,1.852369
2,NaN,NaN,NaN,Algeria,2021-07-11 04:21:43,28.03390,1.659600,145296,3824,100911.0,40561.0,Algeria,331.339895,2.631869
3,NaN,NaN,NaN,Andorra,2021-07-11 04:21:43,42.50630,1.521800,14075,127,13770.0,178.0,Andorra,18216.527535,0.902309
4,NaN,NaN,NaN,Angola,2021-07-11 04:21:43,-11.20270,17.873900,39881,929,34298.0,4654.0,Angola,121.343257,2.329430


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,120216,120216,120216,120216,125937,127464,129021,130113,131586,131586
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132523,132526,132534,132535,132537,132544,132557,132565,132580,132587
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,140075,140550,141007,141471,141966,142447,143032,143652,144483,145296


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,4962,4962,4962,4962,5283,5360,5415,5477,5561,5561
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2456,2456,2456,2456,2456,2456,2456,2456,2456,2456
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3726,3735,3745,3755,3765,3775,3786,3798,3811,3824


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,71924,72700,73635,74480,75314,76218,77280,78521,79446,80226
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,130014,130018,130023,130027,130029,130033,130035,130037,130043,130047
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,97380,97701,98031,98387,98748,99101,99510,99931,100383,100911


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7257,7262,7262,7262,7262,7262,7277,7277,7294,7299
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,22027,22043,22043,22043,22043,22043,22154,22154,22220,22267
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2346,2347,2347,2347,2347,2347,2354,2354,2361,2365


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,315,315,315,315,315,315,316,316,317,318
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,60,60,60,60,60,60,60,60,60,60


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
656,1001.0,Autauga,Alabama,US,2021-07-11 04:21:43,32.539527,-86.644082,7299,113,NaN,NaN,"Autauga, Alabama, US",13064.490147,1.548157
693,1075.0,Lamar,Alabama,US,2021-07-11 04:21:43,33.779950,-88.096680,1478,38,NaN,NaN,"Lamar, Alabama, US",10706.265846,2.571042
694,1077.0,Lauderdale,Alabama,US,2021-07-11 04:21:43,34.901719,-87.656247,9703,252,NaN,NaN,"Lauderdale, Alabama, US",10463.824693,2.597135


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,33847784,607132,0.0
India,30795716,407145,29933538.0
Brazil,19069003,532893,16983333.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,33847784,607132,0.0,33240652.0,2021-07-11 04:21:43,37.950547,-91.419547
India,30795716,407145,29933538.0,455033.0,2021-07-11 04:21:43,23.088275,81.806127
Brazil,19069003,532893,16983333.0,1552777.0,2021-07-11 04:21:43,-12.669522,-48.480493
France,5870463,111511,407372.0,5351580.0,2021-07-11 04:21:43,6.430808,-34.730285
Russia,5688807,139896,5122104.0,426807.0,2021-07-11 04:21:43,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3834068,63932,0.0
Texas,3016891,52624,0.0
Florida,2404895,38157,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3834068,63932,0.0,3770136.0,2021-07-11 04:21:43,37.843962,-120.728594
Texas,3016891,52624,0.0,2964267.0,2021-07-11 04:21:43,31.660643,-98.653069
Florida,2404895,38157,0.0,2366738.0,2021-07-11 04:21:43,28.940755,-82.700744
New York,2119692,53733,0.0,2065959.0,2021-07-11 04:21:43,42.544151,-75.474183
Illinois,1395497,25730,0.0,1369767.0,2021-07-11 04:21:43,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1256557,24542,0.0
Arizona,Maricopa,563897,10339,0.0
Illinois,Cook,557922,10522,0.0
Florida,Miami-Dade,515765,6472,0.0
Texas,Harris,405627,6622,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1256557,24542,0.0,1232015.0,2021-07-11 04:21:43,34.308284,-118.228241,6037.0
Arizona,Maricopa,563897,10339,0.0,553558.0,2021-07-11 04:21:43,33.348359,-112.491815,4013.0
Illinois,Cook,557922,10522,0.0,547400.0,2021-07-11 04:21:43,41.841448,-87.816588,17031.0
Florida,Miami-Dade,515765,6472,0.0,509293.0,2021-07-11 04:21:43,25.611236,-80.551706,12086.0
Texas,Harris,405627,6622,0.0,399005.0,2021-07-11 04:21:43,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-08,130113,132565,143652,14050,39593,1266,4613019,225987,30951,651128,...,5040060,375298,114529,4,281907,24810,314869,6936,170456,62383
2021-07-09,131586,132580,144483,14075,39791,1266,4627537,226135,31015,651291,...,5075945,375902,115011,4,282940,26608,315047,6940,172405,65066
2021-07-10,131586,132587,145296,14075,39881,1266,4639098,226285,31100,651453,...,5107780,376281,115478,4,283742,28470,315107,6941,174789,66853


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-08,5477,2456,3798,127,925,42,97904,4536,910,10721,...,128601,5772,760,1,3253,104,3577,1365,2672,2029
2021-07-09,5561,2456,3811,127,928,42,98148,4540,910,10722,...,128631,5790,764,1,3264,110,3579,1366,2736,2084
2021-07-10,5561,2456,3824,127,929,42,98501,4542,911,10723,...,128665,5807,767,1,3278,112,3580,1366,2775,2126


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-08,78521,130037,99931,13760,34100,1222,4226694,217381,23850,638618,...,15888,360438,110041,3,263830,8950,309251,4119,152103,42855
2021-07-09,79446,130043,100383,13770,34169,1222,4246200,217490,23852,638719,...,15888,361670,110377,3,264921,8984,309543,4128,154776,44021
2021-07-10,80226,130047,100911,13770,34298,1222,4264937,217587,23858,638814,...,15892,362682,110759,3,266231,9204,309689,4135,157489,44580


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/1/21,7/2/21,7/3/21,7/4/21,7/5/21,7/6/21,7/7/21,7/8/21,7/9/21,7/10/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7257,7262,7262,7262,7262,7262,7277,7277,7294,7299
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1476,1476,1476,1476,1476,1476,1478,1478,1478,1478
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9645,9653,9653,9653,9653,9653,9674,9674,9698,9703
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3144,3144,3144,3144,3144,3144,3146,3146,3147,3150
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16332,16343,16343,16343,16343,16343,16390,16390,16422,16448


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-08,552911,71598,0,898908,354305,3829942,560860,349798,109931,49,...,869191,3013216,418321,24445,3970,681890,454650,164399,678459,62865
2021-07-09,554071,71731,0,899829,355460,3832060,561912,349969,109986,49,...,869752,3015520,418976,24457,4005,682147,455103,164465,678604,62966
2021-07-10,554605,71731,0,900636,355460,3834068,562401,349969,110019,49,...,869752,3016891,418976,24457,4005,682433,455103,164465,678604,62966


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-07-08        7277   22154    2354  2699   7013    1250   2270   14795   
2021-07-09        7294   22220    2361  2699   7018    1250   2273   14818   
2021-07-10        7299   22267    2365  2702   7023    1250   2279   14833   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-08         3751     1885  ...    2905    698     3278      800   
2021-07-09         3757     1896  ...    2906    702     3278      804   
2021-07-10         3760     1896  ...    2906    702     3278      804   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-08           4820  3807  2328          0      929    663  
2021-07-09           4833  3816  2329          0      929    664  
2021-07-10           4833  3816  2329          0      929    664  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-08,11387,377,0,18003,5944,63815,6827,8279,1695,0,...,12592,52567,2389,258,32,11441,5986,2904,8158,751
2021-07-09,11398,379,0,18009,5948,63875,6837,8279,1695,0,...,12596,52606,2393,258,32,11448,5997,2908,8182,751
2021-07-10,11402,379,0,18029,5948,63932,6848,8279,1695,0,...,12596,52624,2393,258,32,11450,5997,2908,8182,751


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-07-08         113     316      60   65    139      42     71     331   
2021-07-09         113     317      60   65    139      42     71     331   
2021-07-10         113     318      60   65    139      42     71     331   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-08          123       45  ...      33     13       31        7   
2021-07-09          123       47  ...      33     13       31        7   
2021-07-10          123       47  ...      33     13       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-08             40    11    13          0       26      6  
2021-07-09             40    11    13          0       26      6  
2021-07-10             40    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-08,0.008464,0.000060,0.004335,0.002068,0.002583,0.000791,0.004192,0.000824,0.001553,0.000184,...,0.006409,0.001690,0.004297,0.0,0.003299,0.060936,0.000000,0.000288,0.008597,0.035798
2021-07-09,0.011321,0.000113,0.005785,0.001779,0.005001,0.000000,0.003147,0.000655,0.002068,0.000250,...,0.007120,0.001609,0.004209,0.0,0.003664,0.072471,0.000565,0.000577,0.011434,0.043009
2021-07-10,0.000000,0.000053,0.005627,0.000000,0.002262,0.000000,0.002498,0.000663,0.002741,0.000249,...,0.006272,0.001008,0.004060,0.0,0.002835,0.069979,0.000190,0.000144,0.013828,0.027464


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-08,0.011450,0.0,0.003170,0.0,0.000000,0.0,0.004772,0.001104,0.000000,0.000187,...,0.000280,0.002431,0.002639,0.0,0.004322,0.019608,0.000000,0.000733,0.020626,0.028383
2021-07-09,0.015337,0.0,0.003423,0.0,0.003243,0.0,0.002492,0.000882,0.000000,0.000093,...,0.000233,0.003119,0.005263,0.0,0.003381,0.057692,0.000559,0.000733,0.023952,0.027107
2021-07-10,0.000000,0.0,0.003411,0.0,0.001078,0.0,0.003597,0.000441,0.001099,0.000093,...,0.000264,0.002936,0.003927,0.0,0.004289,0.018182,0.000279,0.000000,0.014254,0.020154


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-08,0.016058,0.000015,0.004231,0.000218,0.001557,0.0,0.004806,0.000359,0.000084,0.000081,...,0.001134,0.003237,0.003328,0.0,0.004389,0.045927,0.000000,0.000486,0.017949,0.012403
2021-07-09,0.011780,0.000046,0.004523,0.000727,0.002023,0.0,0.004615,0.000501,0.000084,0.000158,...,0.000000,0.003418,0.003053,0.0,0.004135,0.003799,0.000944,0.002185,0.017574,0.027208
2021-07-10,0.009818,0.000031,0.005260,0.000000,0.003775,0.0,0.004413,0.000446,0.000252,0.000149,...,0.000252,0.002798,0.003461,0.0,0.004945,0.024488,0.000472,0.001696,0.017529,0.012698


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-08,0.000000,0.000000,NaN,0.000696,0.003427,0.000897,0.000787,0.000157,0.000282,0.0,...,0.000521,0.000760,0.001599,0.000205,0.000000,0.000427,0.001282,0.000322,0.000335,0.002040
2021-07-09,0.002098,0.001858,NaN,0.001025,0.003260,0.000553,0.001876,0.000489,0.000500,0.0,...,0.000645,0.000765,0.001566,0.000491,0.008816,0.000377,0.000996,0.000401,0.000214,0.001607
2021-07-10,0.000964,0.000000,NaN,0.000897,0.000000,0.000524,0.000870,0.000000,0.000300,0.0,...,0.000000,0.000455,0.000000,0.000000,0.000000,0.000419,0.000000,0.000000,0.000000,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-07-08      0.000000  0.000000  0.000000  0.000000  0.000000     0.0   
2021-07-09      0.002336  0.002979  0.002974  0.000000  0.000713     0.0   
2021-07-10      0.000685  0.002115  0.001694  0.001112  0.000712     0.0   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-07-08      0.000000  0.000000  0.000000  0.000000  ...  0.000689   
2021-07-09      0.001322  0.001555  0.001600  0.005836  ...  0.000344   
2021-07-10      0.002640  0.001012  0.000799  0.000000  ...  0.000000   

Province_State                                                             \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton    Uinta   
2021-07-08      0.007215  0.000611    0.000   0.003122  0.000526  0.00043   
2021-07-09      0.005731  0.000000    0.005   0.002697  0.002364  0.00043   
2021-07-10      0.000000  0.000000    0.000   0.000000  0.000000  0.00000   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-07-08            NaN  0.002157  0.000000  
2021-07-09            NaN  0.000000  0.001508  
2021-07-10            NaN  0.000000  0.000000  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-08,0.000000,0.000000,NaN,-0.000056,0.001854,0.001067,0.000440,0.000121,0.0,NaN,...,0.000715,0.000685,0.000838,0.0,0.0,0.000437,0.002176,0.001034,0.000491,0.0
2021-07-09,0.000966,0.005305,NaN,0.000333,0.000673,0.000940,0.001465,0.000000,0.0,NaN,...,0.000318,0.000742,0.001674,0.0,0.0,0.000612,0.001838,0.001377,0.002942,0.0
2021-07-10,0.000351,0.000000,NaN,0.001111,0.000000,0.000892,0.001609,0.000000,0.0,NaN,...,0.000000,0.000342,0.000000,0.0,0.0,0.000175,0.000000,0.000000,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-07-08         0.0  0.000000     0.0  0.0    0.0     0.0    0.0     0.0   
2021-07-09         0.0  0.003165     0.0  0.0    0.0     0.0    0.0     0.0   
2021-07-10         0.0  0.003155     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                     ... Wyoming                           \
Admin2         Chambers  Cherokee  ...    Park Platte Sheridan Sublette   
2021-07-08          0.0  0.000000  ...     0.0    0.0      0.0      0.0   
2021-07-09          0.0  0.044444  ...     0.0    0.0      0.0      0.0   
2021-07-10          0.0  0.000000  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-07-08            0.0   0.0   0.0        NaN      0.0    0.0  
2021-07-09            0.0   0.0   0.0        NaN      0.0    0.0  
2021-07-10            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-08,0.011884,0.000064,0.004040,0.002231,0.002506,0.000497,0.004173,0.000641,0.001378,0.000165,...,0.006198,0.001738,0.004264,5.514537e-25,0.003720,0.054435,0.000173,0.000311,0.009592,0.034970
2021-07-09,0.011603,0.000088,0.004912,0.002005,0.003754,0.000249,0.003660,0.000648,0.001723,0.000208,...,0.006659,0.001674,0.004236,2.757269e-25,0.003692,0.063453,0.000369,0.000444,0.010513,0.038989
2021-07-10,0.005801,0.000071,0.005270,0.001003,0.003008,0.000124,0.003079,0.000656,0.002232,0.000228,...,0.006465,0.001341,0.004148,1.378634e-25,0.003263,0.066716,0.000280,0.000294,0.012170,0.033227


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-08,0.014298,5.285802e-08,0.002975,5.602632e-23,0.002409,3.114617e-17,0.004791,0.000754,3.554647e-30,0.000202,...,0.000253,0.003260,0.003682,0.0,0.004716,0.036425,0.000216,0.000689,0.022142,0.022205
2021-07-09,0.014817,2.642901e-08,0.003199,2.801316e-23,0.002826,1.557309e-17,0.003642,0.000818,1.777324e-30,0.000148,...,0.000243,0.003189,0.004473,0.0,0.004049,0.047059,0.000388,0.000711,0.023047,0.024656
2021-07-10,0.007409,1.321450e-08,0.003305,1.400658e-23,0.001952,7.786543e-18,0.003619,0.000629,5.494505e-04,0.000121,...,0.000254,0.003063,0.004200,0.0,0.004169,0.032620,0.000333,0.000355,0.018651,0.022405


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-07-08,0.014457,0.000019,0.004037,0.000485,0.002408,3.200597e-06,0.004335,0.000311,0.000100,0.000119,...,0.001376,0.003339,0.003139,1.323489e-23,0.004590,0.042024,0.000307,0.000937,0.015800,0.013465
2021-07-09,0.013119,0.000032,0.004280,0.000606,0.002216,1.600298e-06,0.004475,0.000406,0.000092,0.000138,...,0.000688,0.003378,0.003096,6.617445e-24,0.004363,0.022912,0.000626,0.001561,0.016687,0.020337
2021-07-10,0.011468,0.000032,0.004770,0.000303,0.002996,8.001492e-07,0.004444,0.000426,0.000172,0.000144,...,0.000470,0.003088,0.003278,3.308722e-24,0.004654,0.023700,0.000549,0.001628,0.017108,0.016518


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-08,0.000738,0.000765,NaN,0.000617,0.002767,0.000702,0.000724,0.000210,0.000178,2.180993e-145,...,0.000516,0.000828,0.001398,0.000249,0.004858,0.000365,0.001204,0.000367,0.000259,0.001815
2021-07-09,0.001418,0.001311,NaN,0.000821,0.003013,0.000628,0.001300,0.000349,0.000339,1.090496e-145,...,0.000581,0.000796,0.001482,0.000370,0.006837,0.000371,0.001100,0.000384,0.000236,0.001711
2021-07-10,0.001191,0.000656,NaN,0.000859,0.001507,0.000576,0.001085,0.000175,0.000320,5.452482e-146,...,0.000290,0.000626,0.000741,0.000185,0.003418,0.000395,0.000550,0.000192,0.000118,0.000855


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-07-08      0.000525  0.001269  0.000750  0.000562  0.000900  0.000007   
2021-07-09      0.001430  0.002124  0.001862  0.000281  0.000806  0.000004   
2021-07-10      0.001058  0.002120  0.001778  0.000696  0.000759  0.000002   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-07-08      0.000676  0.000290  0.000875  0.001467  ...  0.000342   
2021-07-09      0.000999  0.000922  0.001237  0.003651  ...  0.000343   
2021-07-10      0.001819  0.000967  0.001018  0.001826  ...  0.000172   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-07-08      0.006186  0.000543  0.000626   0.002140  0.000396  0.000651   
2021-07-09      0.005958  0.000272  0.002813   0.002418  0.001380  0.000540   
2021-07-10      0.002979  0.000136  0.001407   0.001209  0.000690  0.000270   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-07-08           -1.0  0.001087  0.000799  
2021-07-09           -1.0  0.000544  0.001154  
2021-07-10           -1.0  0.000272  0.000577  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-07-08,0.000645,0.000016,NaN,0.000205,0.001431,0.000779,0.000529,0.000031,0.000005,NaN,...,0.000541,0.000544,0.001019,0.000046,0.016667,0.000334,0.002088,0.000626,0.000629,0.000679
2021-07-09,0.000806,0.002660,NaN,0.000269,0.001052,0.000860,0.000997,0.000016,0.000002,NaN,...,0.000429,0.000643,0.001346,0.000023,0.008333,0.000473,0.001963,0.001002,0.001785,0.000339
2021-07-10,0.000578,0.001330,NaN,0.000690,0.000526,0.000876,0.001303,0.000008,0.000001,NaN,...,0.000215,0.000493,0.000673,0.000011,0.004167,0.000324,0.000981,0.000501,0.000893,0.000170


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                                  \
Admin2               Autauga   Baldwin       Barbour      Bibb        Blount   
2021-07-08      2.374155e-12  0.000800  2.586235e-07  0.003906  2.035966e-19   
2021-07-09      1.187078e-12  0.001982  1.293118e-07  0.001953  1.017983e-19   
2021-07-10      5.935388e-13  0.002568  6.465589e-08  0.000977  5.089916e-20   

Province_State                                                                \
Admin2               Bullock        Butler   Calhoun  Chambers      Cherokee   
2021-07-08      6.770076e-19  1.675586e-15  0.000758 -0.000016  1.881835e-26   
2021-07-09      3.385038e-19  8.377931e-16  0.000379 -0.000008  2.222222e-02   
2021-07-10      1.692519e-19  4.188966e-16  0.000189 -0.000004  1.111111e-02   

Province_State  ...   Wyoming                                        \
Admin2          ...      Park    Platte      Sheridan      Sublette   
2021-07-08      ...  0.003971  0.000083  8.026341e-37  3.526044e-57   
2021-07-09      ...  0.001986  0.000041  4.013170e-37  1.763022e-57   
2021-07-10      ...  0.000993  0.000021  2.006585e-37  8.815109e-58   

Province_State                                                       \
Admin2            Sweetwater         Teton         Uinta Unassigned   
2021-07-08      1.528324e-09  2.069606e-10  1.129377e-21   0.752941   
2021-07-09      7.641621e-10  1.034803e-10  5.646886e-22   0.752941   
2021-07-10      3.820811e-10  5.174014e-11  2.823443e-22   0.752941   

Province_State                              
Admin2              Washakie        Weston  
2021-07-08      2.190790e-49  1.862645e-10  
2021-07-09      1.095395e-49  9.313226e-11  
2021-07-10      5.476976e-50  4.656613e-11  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
